In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# Import package
import json
import tweepy
import re
import pandas as pd
import numpy as np
import csv
from time import sleep
import psycopg2

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    """
    Custom stream lister class (Inherited from tweepy StreamListener)
    Overriding methods **on_data** and **on_error** will give greater control on data storage and error handling. 
    """
    
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
    
    def on_data(self, tweet_data):
        try:
            #Converting in json format
            tweet_data = json.loads(tweet_data)
            
            # Keys of Json Dict
            tweet_keys = tweet_data.keys()
            
                        
            #Checking for retweeted_status
            if 'retweeted_status' in tweet_keys:
                return True  

                 
            # Database data
            if tweet_data['place'] is not None:
                if 'extended_tweet' in tweet_keys:
                    created_at = tweet_data['created_at'] 
                    tweet_id = tweet_data['id'] 
                    user_id = tweet_data['user']['id'] 
                    tweet_text = str(tweet_data['extended_tweet']['full_text'].replace('&amp;', '&'))
                    city = tweet_data['place']['name'] 
                    country = tweet_data['place']['country_code']
                else:
                    created_at = tweet_data['created_at']
                    tweet_id = tweet_data['id'] 
                    user_id = tweet_data['user']['id']
                    tweet_text = str(tweet_data['text'].replace('&amp;', '&'))
                    city = tweet_data['place']['name']
                    country = tweet_data['place']['country_code']
            else:
                if 'extended_tweet' in tweet_keys:
                    created_at = tweet_data['created_at'] 
                    tweet_id = tweet_data['id'] 
                    user_id = tweet_data['user']['id'] 
                    tweet_text = str(tweet_data['extended_tweet']['full_text'].replace('&amp;', '&'))
                    city = 'nf'
                    country = 'nf'
                else:
                    created_at = tweet_data['created_at']
                    tweet_id = tweet_data['id']
                    user_id = tweet_data['user']['id'] 
                    tweet_text = str(tweet_data['text'].replace('&amp;', '&')) 
                    city = 'nf'
                    country = 'nf'
                
            # Writing data to database
            #print("INSERT INTO twitter_drug (tweet_id, user_id, tweet_text, city, country, created_at) VALUES (%d, %d, %s, %s, %s, %s)"%(tweet_id, user_id, tweet_text, city, country, created_at))
            cur.execute("INSERT INTO twitter_drug (tweet_id, user_id, tweet_text, city, country, created_at) VALUES (%s, %s, %s, %s, %s, %s)",(tweet_id, user_id, tweet_text, city, country, created_at))
            conn.commit()
    
            return True    
            
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            sleep(10)
        return True
        
    def on_error(self, status):
        print(status)
        if status == 420:
            return False

In [ ]:
#Twitter API secrets
access_token = "3*********-tb6**********GJUdLKfis84S*******IhTTUK"
access_token_secret = "m6*********sXImDQHZrb********jRZSn5jAjOS"
consumer_key = "J77p***********dUHJpwffm"
consumer_secret = "12zj***********TGDJP3heHzfDi*******tFKm4x55pTpY6WFXGn"

# Database Information 
db_host = '**.**.***.***'
db_name = 'yu****'
user = 'postgres'
password = 'yu*******'
port = 5433
dsn = "host={} dbname={} user={} password={} port={}".format(db_host, db_name, user, password, port)

In [ ]:
if __name__ == '__main__':
    #Drug list to track
    drug_list = ['ACTAVIS', 'ADDERALL', 'ADVACEF', 'ADVAQUIN', 'ADVIL']
    
    # Athorization 
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token,access_token_secret)

    #connection to postgres
    with psycopg2.connect(dsn) as conn:
        # cursir objext
        with conn.cursor() as cur:
            
            #Instance of stream listener
            l = MyStreamListener()
            stream = tweepy.Stream(auth, l)
            stream.filter(track = drug_list, languages = ['en'])